### The commands which is useful when GDAL packages is used

## Import shapefile into the DB

In [ ]:
ogr2ogr -f PostgreSQL -t_srs EPSG:4326 PG:"dbname='postgis' user='behzad' password='gis'" -lco SCHEMA=chp01 countries.shp -nlt MULTIPOLYGON -lco OVERWRITE=YES -nln africa_countries -lco GEOMETRY_NAME=the_geom -where "satellite='T'"

## Updating the the table using the exist shapefile

In [ ]:
ogr2ogr -append -update -f PostgreSQL -t_srs EPSG:3857 PG:"dbname='postgis' user='behzad' password='gis'" -nln chp01.btest -nlt POINT new_con.shp

## Exporting Shapefile from DB (Other formats is also possible)

In [ ]:
ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:4326 new_con.shp PG:"dbname='postgis' user='behzad' password='gis'" -sql " select * from qlayer"

## Information about Table in DB

In [ ]:
ogrinfo PG:"dbname='postgis' user='postgres' password='gis'" chp01.test_table -fid 1

## Information about Raster

### Outside  DB

In [ ]:
gdalinfo tmax9.bil

### inside  DB

In [ ]:
gdalinfo PG:"host=localhost port=5432 dbname=postgis user=behzad password=gis schema='chp01' table='tmax01' mode=1" -sd 1

## Import each raster as a different raster band to DB

In [ ]:
gdalbuildvrt -separate tmax_2012.vrt worldclim/tmax*.bil

In [ ]:
raster2pgsql -d -I -C -M -F -t 100x100 -s 4326 tmax_2012.vrt chp01.tmax_2012_multi > tmax_2012_multi.sql

In [ ]:
psql -d postgis_cookbook -U me -f tmax_2012_multi.sql

## Checking the whether format is supported by GDAL or not

In [ ]:
gdalinfo --formats | grep -i postgisb

## Exporting Raster from DB

### Export specific bands

In [ ]:
gdal_translate -b 1 -b 2 -b 3 -b 4 -b 5 -b 6 PG:"host=localhost port=5432 dbname='postgis' user='postgres' password='gis' schema='chp01' table='tmax_2012_multi' mode='2'" tmax_2012_multi_123456.tif

### Export all of the bands, but only for the geographic area containing Italy

In [ ]:
SELECT ST_Extent(the_geom)
FROM chp01.countries 
WHERE name = 'Italy';

In [ ]:
gdal_translate -projwin 6.619 47.095 18.515 36.649 PG:"host=localhost port=5432 dbname='postgis' user='postgres' password='gis' schema='chp01' table='tmax_2012_multi' mode='2'" tmax_2012_multi.tif

# Re-projection, Sampling, Exporting raster

In [ ]:
gdalwarp -t_srs EPSG:4326 -tr 0.10 0.10 -r bilinear PG:"host=localhost port=5432 dbname='postgis' user='postgres' password='gis' schema='chp01' table='tmax_2012_multi' mode='2'" tmax_2012_multi.tif